In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2
import subprocess
import open3d as o3d
import matplotlib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create a new client instance
client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = bp_lib.find('vehicle.dodge.charger')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [3]:
# Start the traffic light controller in the background
traffic_light_process = subprocess.Popen(['python', 'traffic_light_controller.py'])
traffic_process = subprocess.Popen(['python', 'generate_traffic.py', '--number-of-vehicles', '30', '--number-of-walkers', '50'])
time.sleep(5)

# Set up traffic manager
traffic_manager = client.get_trafficmanager()
vehicle.set_autopilot(True, traffic_manager.get_port())

In [4]:
VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:, :3]  # Remove alpha channel

camera_init_trans = carla.Transform(carla.Location(x=0.0, y=0.0, z=1.6), carla.Rotation(pitch=0.0))

def set_camera_attributes(camera_bp):
    camera_bp.set_attribute('image_size_x', '800')
    camera_bp.set_attribute('image_size_y', '600')
    camera_bp.set_attribute('fov', '90')

# RGB Camera
rgb_camera_bp = bp_lib.find('sensor.camera.rgb')
set_camera_attributes(rgb_camera_bp)
rgb_camera = world.spawn_actor(rgb_camera_bp, camera_init_trans, attach_to=vehicle)

# Semantic Segmentation Camera
sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
set_camera_attributes(sem_camera_bp)
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

# Instance Segmentation Camera
inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation')
set_camera_attributes(inst_camera_bp)
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=vehicle)

# Depth Camera
depth_camera_bp = bp_lib.find('sensor.camera.depth')
set_camera_attributes(depth_camera_bp)
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)

In [5]:
# Prepare image data storage (define sensor_data here so it’s available everywhere)
image_w = 800
image_h = 600
sensor_data = {
    'rgb_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'sem_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'inst_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
    'depth_image': np.zeros((image_h, image_w, 4), dtype=np.uint8),
}

def add_label(image, label, position=(10, 50), font_scale=0.8, color=(255, 255, 255)):
    """Adds a text label to an image."""
    labeled_image = image.copy()
    cv2.putText(labeled_image, label, position, cv2.FONT_HERSHEY_SIMPLEX, 
                font_scale, color, 2, cv2.LINE_AA)
    return labeled_image

def rgb_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['rgb_image'] = add_label(img, "RGB Camera")

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['sem_image'] = add_label(img, "Semantic Segmentation")

def inst_callback(image, data_dict):
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['inst_image'] = add_label(img, "Instance Segmentation")

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    data_dict['depth_image'] = add_label(img, "Depth Camera")

# Start camera listeners
rgb_camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))

In [6]:
lidar_bp = bp_lib.find('sensor.lidar.ray_cast_semantic')
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-25.0')
lidar_bp.set_attribute('channels', '64')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '500000')
lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)

# Radar Sensor
radar_bp = bp_lib.find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '30.0')
radar_bp.set_attribute('vertical_fov', '30.0')
radar_bp.set_attribute('points_per_second', '10000')
radar_init_trans = carla.Transform(carla.Location(z=2))
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle)

# Initialize Open3D point clouds for LiDAR and Radar
lidar_pcd = o3d.geometry.PointCloud()
radar_pcd = o3d.geometry.PointCloud()

def get_random_color():
    color = np.random.rand(3)
    while np.linalg.norm(color) < 0.3:
        color = np.random.rand(3)
    return color

def lidar_callback(point_cloud, pcd):
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    if data.shape[0] == 0:
        return
    data = np.reshape(data, (-1, 6))  # 6 values per point: x, y, z, intensity, object_id, timestamp
    points = data[:, :3]
    colors = np.array([get_random_color() for _ in range(points.shape[0])])
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

def radar_callback(data, pcd):
    radar_data = np.zeros((len(data), 4))
    for i, detection in enumerate(data):
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)
        radar_data[i, :] = [x, y, z, detection.velocity]
    intensity = np.abs(radar_data[:, -1])
    intensity_safe = np.clip(intensity, 1e-6, None)
    intensity_col = 1.0 - np.log(intensity_safe) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])
    ]
    points = radar_data[:, :3]
    points[:, :1] = -points[:, :1]  # flip x-axis if needed
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(int_color)

# Start LiDAR and Radar listeners
lidar.listen(lambda data: lidar_callback(data, lidar_pcd))
radar.listen(lambda data: radar_callback(data, radar_pcd))

In [7]:
cv2.namedWindow('All Cameras', cv2.WINDOW_NORMAL)
cv2.resizeWindow('All Cameras', 1280, 960)
cv2.namedWindow('Duplicate RGB', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Duplicate RGB', 800, 600)

def preprocess_image(img):
    """Ensure the image has 3 channels (drop alpha if present)."""
    if img.shape[2] == 4:
        img = img[:, :, :3]
    return img


vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Carla Lidar', width=960, height=540, left=400, top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True

def add_open3d_axis(vis):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [0.1, 0.0, 0.0],
        [0.0, 0.1, 0.0],
        [0.0, 0.0, 0.1]
    ]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]
    ]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ]))
    vis.add_geometry(axis)

add_open3d_axis(vis)

lidar_added = False
radar_added = False

while True:
    try:
        # --- Process and display camera images ---
        processed_data = {key: preprocess_image(img) for key, img in sensor_data.items()}
        # Check if the images have been updated (nonzero height)
        if all(img.shape[0] > 0 for img in processed_data.values()):
            rgb_clean = processed_data['rgb_image'].copy()
            top_row = np.concatenate([processed_data['rgb_image'], processed_data['sem_image']], axis=1)
            bottom_row = np.concatenate([processed_data['depth_image'], processed_data['inst_image']], axis=1)
            tiled = np.concatenate((top_row, bottom_row), axis=0)
            cv2.imshow('All Cameras', tiled)
            cv2.imshow('Duplicate RGB', rgb_clean)
        
        # --- Update Open3D visualizer with LiDAR and Radar data ---
        if not lidar_added:
            vis.add_geometry(lidar_pcd)
            lidar_added = True
        if not radar_added:
            vis.add_geometry(radar_pcd)
            radar_added = True
        vis.update_geometry(lidar_pcd)
        vis.update_geometry(radar_pcd)
        vis.poll_events()
        vis.update_renderer()

        if cv2.waitKey(1) == ord('q'):
            break
        
        time.sleep(0.005)

    except Exception as e:
        print("Error:", e)
        break

cv2.destroyAllWindows()
vis.destroy_window()

In [8]:
# # Stop and destroy sensors
# lidar.stop()
# lidar.destroy()
# radar.stop()
# radar.destroy()
# rgb_camera.stop()
# rgb_camera.destroy()
# sem_camera.stop()
# sem_camera.destroy()
# inst_camera.stop()
# inst_camera.destroy()
# depth_camera.stop()
# depth_camera.destroy()